In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
with open('../../../data/messages_many_2/messages_many.pkl', 'rb') as file:

    mes_many = pickle.load(file)

In [3]:
import re
def add_name_wo_qutes(str):
    res = []
    name_without_quotes  = re.findall(r"\"(.*?)\"", str)
    for x in name_without_quotes:
        res.append(x)
    return ''.join(res)

In [4]:
issuers = pd.read_csv('../../../data/issuers - issuers.csv', index_col = 1, dtype=str).drop('ID человека', axis= 1)
issuers = issuers.apply(lambda x: x.str.strip())

In [5]:
issuers['name_wo_qutes'] = issuers['EMITENT_FULL_NAME'].apply(add_name_wo_qutes)

In [39]:
tmp = issuers.loc[:, ~issuers.columns.isin(['BGTicker', 'BGTicker.1', 'VeryOddCompany'])].values.flatten()
names = [str(x).strip() for x in tmp.tolist() if str(x) != 'nan' and x != '']
names = list(set(names))
names = [x.lower() for x in names]
del tmp

In [40]:
tmp = issuers[['BGTicker', 'BGTicker.1', 'VeryOddCompany']].values.flatten()
tickers = [str(x)[0:4].strip() for x in tmp.tolist() if str(x) != 'nan' and x != '']
tickers = list(set(tickers))
del tmp

In [8]:
mentions = pd.read_csv('../../../data/sentiment_data_3/mentions.csv', index_col = 0)
mentions = mentions[mentions['issuerid'] > 0]
mentions = mentions.rename(columns={'messageid': 'MessageID'})

In [9]:
sentiment = pd.read_csv('../../../data/sentiment_data_3/sentiment.csv', index_col = 0)
#sentiment = sentiment[sentiment['issuerid'] > 0]
sentiment = sentiment[sentiment['SentimentScore'] != 0]

In [10]:
tmp = sentiment.copy()
tmp['issuerid'] = sentiment['issuerid']
tmp = tmp.groupby(['ChannelID', 'MessageID']).apply(lambda x: list(zip(x['issuerid'].to_list(), x['SentimentScore'].to_list()))).reset_index()
mes_many_issuers_sentiment  = mes_many.merge(tmp, on = ['ChannelID', 'MessageID'], how ='right')
mes_many_issuers_sentiment = mes_many_issuers_sentiment.dropna()
mes_many_issuers_sentiment = mes_many_issuers_sentiment.rename(columns={0: 'sentiment'})

In [11]:
mes_many_issuers_sentiment

,MessageID,ChannelID,DateAdded,DatePosted,MessageText,IsForward,sentiment
0,103,1001029560,2021-02-06 00:52:00,2016-01-14 07:15:26,http://bcs-express.ru/novosti-i-analitika/prog...,False,"[(-1, 3)]"
1,513,1001029560,2021-02-06 00:52:00,2016-06-09 09:47:40,http://bcs-express.ru/novosti-i-analitika/divi...,False,"[(-1, 3)]"
2,6495,1001029560,2021-02-06 00:52:00,2020-09-24 09:46:38,Группа Mail.ru планирует привлечь $600 млн для...,False,"[(-1, 3)]"
3,7869,1001029560,2021-08-11 02:02:52,2021-08-10 15:58:19,Рыночная стоимость Сбербанка превысила $100 мл...,False,"[(-1, 3)]"
4,12838,1001029560,2023-01-23 20:37:25,2023-01-23 15:02:06,​​🎉 Конкурс — Норильский никель после отчетнос...,False,"[(53, 3)]"
...,...,...,...,...,...,...,...
7450,1627,1565800335,2023-06-16 13:23:06,2023-06-16 07:30:00,​🔎 АФК Система (AFKS) - вот кто может оказатьс...,False,"[(204, 3)]"
7451,1632,1565800335,2023-06-19 13:25:14,2023-06-19 07:30:15,​​💎 Алроса (ALRS) - актуализация взгляда на фо...,False,"[(4, 5), (127, 4)]"
7452,1641,1565800335,2023-06-22 13:24:38,2023-06-22 08:32:09,"""​🔥 Новатэк (NVTK) - новые данные с ПМЭФ и про...",False,"[(115, 2)]"
7453,1664,1565800335,2023-07-03 19:32:23,2023-07-03 15:24:56,"⚡ Идеи, которые еще не потеряли актуальность ...",False,"[(12, 4), (4, 4), (111, 4), (53, 4), (58, 4)]"


In [13]:
tmp = mentions.copy()
tmp['issuerid'] = mentions['issuerid']
tmp = tmp.groupby(['ChannelID', 'MessageID'])['issuerid'].apply(lambda x: x.to_list()).reset_index()
mes_many_mentions  = mes_many.merge(tmp, on = ['ChannelID', 'MessageID'], how ='right')
mes_many_mentions = mes_many_mentions.dropna()
#mes_many_mentions['issuerid'] = mes_many_mentions['issuerid'].str.split('+')

In [14]:
mes_many_mentions

,MessageID,ChannelID,DateAdded,DatePosted,MessageText,IsForward,issuerid
0,1113,1001029560,2021-02-06 00:52:00,2017-01-25 14:53:12,У «Ростелекома» вышло приложение «Аллё». Через...,False,[32]
1,1177,1001029560,2021-02-06 00:52:00,2017-02-09 10:01:09,Встречаем! Новая бумага на российском рынке ак...,False,[62]
2,1501,1001029560,2021-02-06 00:52:00,2017-05-04 06:02:56,Несколько мыслей о текущей ситуации на рынке и...,False,[26]
3,2100,1001029560,2021-02-06 00:52:00,2017-10-12 16:19:28,13 ноября состоится пересмотр индекса MSCI Rus...,False,[142]
4,2160,1001029560,2021-02-06 00:52:00,2017-10-24 09:04:06,Анонс ключевых событий: В России - Инвестици...,False,"[111, 7]"
...,...,...,...,...,...,...,...
16166,1627,1565800335,2023-06-16 13:23:06,2023-06-16 07:30:00,​🔎 АФК Система (AFKS) - вот кто может оказатьс...,False,[204]
16167,1632,1565800335,2023-06-19 13:25:14,2023-06-19 07:30:15,​​💎 Алроса (ALRS) - актуализация взгляда на фо...,False,"[4, 127]"
16168,1641,1565800335,2023-06-22 13:24:38,2023-06-22 08:32:09,"""​🔥 Новатэк (NVTK) - новые данные с ПМЭФ и про...",False,[115]
16169,1664,1565800335,2023-07-03 19:32:23,2023-07-03 15:24:56,"⚡ Идеи, которые еще не потеряли актуальность ...",False,"[58, 12, 4, 53, 111]"


In [15]:
import re
def get_names(id):
    res = issuers.loc[id].dropna().to_list()
    name_without_quotes  = re.findall(r"\"(.*?)\"", res[0])
    for x in name_without_quotes:
        res.append(x)
    return res

In [16]:
def name_to_id(name):
    tmp = issuers[(issuers == name).any(axis=1)]
    if not tmp.empty:
        return tmp.index[0]
    else:
        return -100

In [17]:
name_to_id('NMTP')

253

In [21]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from tqdm import tqdm
tqdm.pandas()

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [23]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

def lemmatize(text):
    words = []
    text = str(text)
 
    """# split the value
    tokens = val.split()
     
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()"""

    doc = Doc(text)
    doc.segment(segmenter)

    doc.tag_morph(morph_tagger)

    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    words += [_.lemma for _ in doc.tokens]

    return words

In [69]:
def find_spans(text, regex):
    matches = re.finditer(regex, text)
    return [x.span()  for x in matches]

def find_text(text, regex):
    matches = re.finditer(regex, text)
    return [x.group() for x in matches]

def find_text_spans(text, regex):
    matches = re.finditer(regex, text)
    return [{'str': x.group(), 'lemma': ' '.join(lemmatize(x.group())), 'start': x.span()[0], 'end': x.span()[1] } for x in matches]

In [25]:
mes_many_mentions['MessageText_lemmatized'] = mes_many_mentions['MessageText'].parallel_apply(lambda x: " ".join(lemmatize(x))+" ")

In [59]:
"(" + ")|(".join(tickers) + ")"

'(KMTZ)|(RUAL)|(CBOM)|(PRFN)|(SFIN)|(MGNZ)|(MRKK)|(RLMN)|(ASSB)|(OPIN)|(nmtp)|(RUGR)|(IGST)|(TCS)|(POSI)|(WUSH)|(lsng)|(KLSB)|(CNTL)|(BLNG)|(mrkv)|(MRKS)|(RNFT)|(GLPR)|(LVHK)|(GLTR)|(MRKU)|(FEES)|(MAIL)|(UWGN)|(DIOD)|(IRAO)|(VTBR)|(TRCN)|(HIMC)|(SMLT)|(URKZ)|(IDVP)|(BELU)|(MOEX)|(TRNF)|(PRMB)|(BSPB)|(TNSE)|(VSMO)|(CHEP)|(VZRZ)|(MGTS)|(KGKC)|(FIVE)|(ELTZ)|(RTSB)|(GAZS)|(MISB)|(EELT)|(PIKK)|(FESH)|(TORS)|(LSRG)|(KAZT)|(ZILL)|(KMAZ)|(MFON)|(MDMG)|(HYDR)|(AMEZ)|(AKRN)|(ENPG)|(MAGN)|(RASP)|(KUNF)|(OKEY)|(SFTL)|(ROLO)|(GMKN)|(TGKD)|(SGZH)|(KTSB)|(TGKN)|(TRFM)|(AFLT)|(AFKS)|(ROSN)|(MTSS)|(kube)|(SIBN)|(SVAV)|(DZRD)|(MGNT)|(AGRO)|(UNAC)|(NKHP)|(ALNU)|(PLZL)|(UPRO)|(BANE)|(LPSB)|(SELG)|(KBTK)|(MRKY)|(FLOT)|(KMEZ)|(POLY)|(GTSS)|(IRKT)|(UNKL)|(TGKB)|(YNDX)|(TTLK)|(KUZB)|(mstt)|(SBER)|(LNTA)|(GTRK)|(APTK)|(DSKY)|(FIXP)|(DERZ)|(IRGZ)|(SPBE)|(TGKA)|(CKPZ)|(paza)|(PHOR)|(KSGR)|(CHMK)|(RKKE)|(RUSI)|(GRNT)|(UTAR)|(nomp)|(TUZA)|(KRSB)|(YAKG)|(NVTK)|(LIFE)|(RUSP)|(NPOF)|(ENPL)|(GAZA)|(SNGS)|(CHMF)|(MTLR)

In [55]:
combined = "(" + ")|(".join(tickers) + ")"

mes_many_mentions['tickets_spans'] = mes_many_mentions['MessageText'].parallel_apply(lambda x : find_text_spans(x, combined) )

In [57]:
mes_many_mentions

,MessageID,ChannelID,DateAdded,DatePosted,MessageText,IsForward,issuerid,MessageText_lemmatized,tickets_spans
0,1113,1001029560,2021-02-06 00:52:00,2017-01-25 14:53:12,У «Ростелекома» вышло приложение «Аллё». Через...,False,[32],у « ростелеком » выйти приложение « алле » . ч...,[]
1,1177,1001029560,2021-02-06 00:52:00,2017-02-09 10:01:09,Встречаем! Новая бумага на российском рынке ак...,False,[62],встречать ! новый бумага на российский рынок а...,[]
2,1501,1001029560,2021-02-06 00:52:00,2017-05-04 06:02:56,Несколько мыслей о текущей ситуации на рынке и...,False,[26],несколько мысль о текущий ситуация на рынок и ...,[]
3,2100,1001029560,2021-02-06 00:52:00,2017-10-12 16:19:28,13 ноября состоится пересмотр индекса MSCI Rus...,False,[142],13 ноябрь состояться пересмотр индекс msci rus...,[]
4,2160,1001029560,2021-02-06 00:52:00,2017-10-24 09:04:06,Анонс ключевых событий: В России - Инвестици...,False,"[111, 7]",анонс ключевой событие : в россия - инвестицио...,[]
...,...,...,...,...,...,...,...,...,...
16166,1627,1565800335,2023-06-16 13:23:06,2023-06-16 07:30:00,​🔎 АФК Система (AFKS) - вот кто может оказатьс...,False,[204],​🔎 афк система ( afks ) - вот кто мочь оказать...,"[{'str': 'AFKS', 'lemma': 'afks', 'start': 16,..."
16167,1632,1565800335,2023-06-19 13:25:14,2023-06-19 07:30:15,​​💎 Алроса (ALRS) - актуализация взгляда на фо...,False,"[4, 127]",​​💎 алрос ( alrs ) - актуализация взгляд на фо...,"[{'str': 'ALRS', 'lemma': 'alrs', 'start': 12,..."
16168,1641,1565800335,2023-06-22 13:24:38,2023-06-22 08:32:09,"""​🔥 Новатэк (NVTK) - новые данные с ПМЭФ и про...",False,[115],""" ​🔥 новатэк ( nvtk ) - новый данные с пмэф и ...","[{'str': 'NVTK', 'lemma': 'nvtk', 'start': 13,..."
16169,1664,1565800335,2023-07-03 19:32:23,2023-07-03 15:24:56,"⚡ Идеи, которые еще не потеряли актуальность ...",False,"[58, 12, 4, 53, 111]","⚡ идея , который еще не потерять актуальность ...","[{'str': 'ENPG', 'lemma': 'enpg', 'start': 421..."


In [58]:
"\b(" + ")|\b(".join(names) + ")"

'\x08(lenzoloto)|\x08(пао чмк)|\x08(меридиан)|\x08(transcontainer)|\x08(левенгук)|\x08(новатэк)|\x08(gas-service)|\x08(diod plant of ecological technics and ecomeals)|\x08(npofizika)|\x08(публичное акционерное общество научно-производственное объединение "наука")|\x08(акрон)|\x08(ик русс-инвест)|\x08(публичное акционерное общество энергетики и электрификации "сахалинэнерго")|\x08(tattelecom)|\x08(публичное акционерное общество "одк-сатурн")|\x08(headhunter group plc (хэдхантер групп пиэлси) (эмитент депозитарных расписок - jpmorgan chase bank, n.a. (джипиморган чейз банк, эн.эй)))|\x08(пао химпром)|\x08(яндекс)|\x08("российский акционерный коммерческий дорожный банк" (публичное акционерное общество))|\x08(rusal)|\x08(positive tech)|\x08(м.видео)|\x08(gazprom neft)|\x08(челябинский металлургический комбинат)|\x08(алроса-нюрба)|\x08(lpsbg)|\x08(пао электроцинк)|\x08(публичное акционерное общество "сургутнефтегаз")|\x08(новороссийский комбинат хлебопродуктов)|\x08(пао россети)|\x08(икс 5 

In [71]:
combined = "(" + ")|(".join(names) + ")"

mes_many_mentions['names_spans'] = np.nan

mes_many_mentions['names_spans'] = mes_many_mentions['MessageText_lemmatized'].parallel_apply(lambda x : find_text_spans(x, combined) )

In [72]:
mes_many_mentions

,MessageID,ChannelID,DateAdded,DatePosted,MessageText,IsForward,issuerid,MessageText_lemmatized,tickets_spans,names_spans
0,1113,1001029560,2021-02-06 00:52:00,2017-01-25 14:53:12,У «Ростелекома» вышло приложение «Аллё». Через...,False,[32],у « ростелеком » выйти приложение « алле » . ч...,[],"[{'str': 'ростелеком', 'lemma': 'ростелеком', ..."
1,1177,1001029560,2021-02-06 00:52:00,2017-02-09 10:01:09,Встречаем! Новая бумага на российском рынке ак...,False,[62],встречать ! новый бумага на российский рынок а...,[],"[{'str': 'ор', 'lemma': 'ор', 'start': 54, 'en..."
2,1501,1001029560,2021-02-06 00:52:00,2017-05-04 06:02:56,Несколько мыслей о текущей ситуации на рынке и...,False,[26],несколько мысль о текущий ситуация на рынок и ...,[],"[{'str': 'афк система', 'lemma': 'афк система'..."
3,2100,1001029560,2021-02-06 00:52:00,2017-10-12 16:19:28,13 ноября состоится пересмотр индекса MSCI Rus...,False,[142],13 ноябрь состояться пересмотр индекс msci rus...,[],"[{'str': 'полюс', 'lemma': 'полюс', 'start': 9..."
4,2160,1001029560,2021-02-06 00:52:00,2017-10-24 09:04:06,Анонс ключевых событий: В России - Инвестици...,False,"[111, 7]",анонс ключевой событие : в россия - инвестицио...,[],"[{'str': 'ор', 'lemma': 'ор', 'start': 52, 'en..."
...,...,...,...,...,...,...,...,...,...,...
16166,1627,1565800335,2023-06-16 13:23:06,2023-06-16 07:30:00,​🔎 АФК Система (AFKS) - вот кто может оказатьс...,False,[204],​🔎 афк система ( afks ) - вот кто мочь оказать...,"[{'str': 'AFKS', 'lemma': 'afks', 'start': 16,...","[{'str': 'афк система', 'lemma': 'афк система'..."
16167,1632,1565800335,2023-06-19 13:25:14,2023-06-19 07:30:15,​​💎 Алроса (ALRS) - актуализация взгляда на фо...,False,"[4, 127]",​​💎 алрос ( alrs ) - актуализация взгляд на фо...,"[{'str': 'ALRS', 'lemma': 'alrs', 'start': 12,...","[{'str': 'ор', 'lemma': 'ор', 'start': 401, 'e..."
16168,1641,1565800335,2023-06-22 13:24:38,2023-06-22 08:32:09,"""​🔥 Новатэк (NVTK) - новые данные с ПМЭФ и про...",False,[115],""" ​🔥 новатэк ( nvtk ) - новый данные с пмэф и ...","[{'str': 'NVTK', 'lemma': 'nvtk', 'start': 13,...","[{'str': 'новатэк', 'lemma': 'новатэк', 'start..."
16169,1664,1565800335,2023-07-03 19:32:23,2023-07-03 15:24:56,"⚡ Идеи, которые еще не потеряли актуальность ...",False,"[58, 12, 4, 53, 111]","⚡ идея , который еще не потерять актуальность ...","[{'str': 'ENPG', 'lemma': 'enpg', 'start': 421...","[{'str': 'ор', 'lemma': 'ор', 'start': 12, 'en..."


In [73]:
issuers['comb_all_names'] = issuers.apply(lambda row: ' '.join(row.dropna().values).lower(), axis=1)

In [74]:
def match_as_substr(NER_list, issuers):

    id_list = []
    for token in NER_list:

        id_list_lemma = issuers['comb_all_names'].index[issuers['comb_all_names'].str.lower().str.find(token['lemma']) != -1].to_list()
        id_list_str = issuers['comb_all_names'].index[issuers['comb_all_names'].str.lower().str.find(token['str'].lower()) != -1].to_list()

        id_list += list(set(id_list_lemma + id_list_str))


    return [(x,- 1) for x in list(set(id_list))]


def compute_diff(true_list, pred_list):
    true_list_ = [int(x[0]) for x in true_list]
    pred_list_ = [int(x[0]) for x in pred_list]
    
    return len(set(true_list_) - set(pred_list_))/len(true_list_)

def remove_or(ner_list):
    res = []
    for x in ner_list:
        if x['str'] != 'ор':
            res += [x]
    return res

In [97]:
mes_many_mentions['names_spans'] = mes_many_mentions['names_spans'].progress_apply(remove_or)

100%|██████████| 14561/14561 [00:00<00:00, 566379.43it/s]


In [98]:
mes_many_mentions['matched_ORG'] = mes_many_mentions['names_spans'].progress_apply(lambda x : match_as_substr(x, issuers))

100%|██████████| 14561/14561 [00:51<00:00, 284.05it/s]


In [103]:
mes_many_mentions.issuerid = mes_many_mentions.issuerid.apply(lambda list_in : [(x, -1) for x in list_in])

In [104]:
mes_many_mentions

,MessageID,ChannelID,DateAdded,DatePosted,MessageText,IsForward,issuerid,MessageText_lemmatized,tickets_spans,names_spans,matched_ORG
0,1113,1001029560,2021-02-06 00:52:00,2017-01-25 14:53:12,У «Ростелекома» вышло приложение «Аллё». Через...,False,"[(32, -1)]",у « ростелеком » выйти приложение « алле » . ч...,[],"[{'str': 'ростелеком', 'lemma': 'ростелеком', ...","[(142, -1)]"
1,1177,1001029560,2021-02-06 00:52:00,2017-02-09 10:01:09,Встречаем! Новая бумага на российском рынке ак...,False,"[(62, -1)]",встречать ! новый бумага на российский рынок а...,[],"[{'str': 'детский мир', 'lemma': 'детский мир'...","[(62, -1)]"
2,1501,1001029560,2021-02-06 00:52:00,2017-05-04 06:02:56,Несколько мыслей о текущей ситуации на рынке и...,False,"[(26, -1)]",несколько мысль о текущий ситуация на рынок и ...,[],"[{'str': 'афк система', 'lemma': 'афк система'...","[(26, -1)]"
3,2100,1001029560,2021-02-06 00:52:00,2017-10-12 16:19:28,13 ноября состоится пересмотр индекса MSCI Rus...,False,"[(142, -1)]",13 ноябрь состояться пересмотр индекс msci rus...,[],"[{'str': 'полюс', 'lemma': 'полюс', 'start': 9...","[(32, -1), (26, -1), (142, -1), (90, -1), (91,..."
4,2160,1001029560,2021-02-06 00:52:00,2017-10-24 09:04:06,Анонс ключевых событий: В России - Инвестици...,False,"[(111, -1), (7, -1)]",анонс ключевой событие : в россия - инвестицио...,[],"[{'str': 'втб', 'lemma': 'втб', 'start': 57, '...","[(236, -1), (111, -1), (174, -1), (7, -1)]"
...,...,...,...,...,...,...,...,...,...,...,...
16166,1627,1565800335,2023-06-16 13:23:06,2023-06-16 07:30:00,​🔎 АФК Система (AFKS) - вот кто может оказатьс...,False,"[(204, -1)]",​🔎 афк система ( afks ) - вот кто мочь оказать...,"[{'str': 'AFKS', 'lemma': 'afks', 'start': 16,...","[{'str': 'афк система', 'lemma': 'афк система'...","[(18, -1), (26, -1), (204, -1), (100, -1)]"
16167,1632,1565800335,2023-06-19 13:25:14,2023-06-19 07:30:15,​​💎 Алроса (ALRS) - актуализация взгляда на фо...,False,"[(4, -1), (127, -1)]",​​💎 алрос ( alrs ) - актуализация взгляд на фо...,"[{'str': 'ALRS', 'lemma': 'alrs', 'start': 12,...","[{'str': 'алроса', 'lemma': 'алрос', 'start': ...","[(27, -1), (4, -1), (127, -1)]"
16168,1641,1565800335,2023-06-22 13:24:38,2023-06-22 08:32:09,"""​🔥 Новатэк (NVTK) - новые данные с ПМЭФ и про...",False,"[(115, -1)]",""" ​🔥 новатэк ( nvtk ) - новый данные с пмэф и ...","[{'str': 'NVTK', 'lemma': 'nvtk', 'start': 13,...","[{'str': 'новатэк', 'lemma': 'новатэк', 'start...","[(160, -1), (66, -1), (39, -1), (44, -1), (45,..."
16169,1664,1565800335,2023-07-03 19:32:23,2023-07-03 15:24:56,"⚡ Идеи, которые еще не потеряли актуальность ...",False,"[(58, -1), (12, -1), (4, -1), (53, -1), (111, ...","⚡ идея , который еще не потерять актуальность ...","[{'str': 'ENPG', 'lemma': 'enpg', 'start': 421...","[{'str': 'норникель', 'lemma': 'норникель', 's...","[(160, -1), (165, -1), (166, -1), (199, -1), (..."


In [105]:
mes_many_mentions[['issuerid', 'matched_ORG']].progress_apply(lambda x : compute_diff(x['issuerid'], x['matched_ORG']), axis= 1).mean()

100%|██████████| 14561/14561 [00:00<00:00, 158101.28it/s]


0.2902908119021266

In [ ]:
mes_many_issuers[['MessageText', 'issuerid', 'tickets_spans', 'names_spans']].appply(lambda row:  , axis = 1)

,issuerid,tickets_spans,names_spans
0,[32],[],"[(3, 13)]"
1,[62],[],[]
2,[26],[],"[(76, 87)]"
3,[142],[],"[(96, 101), (103, 106), (108, 115), (124, 132)..."
4,"[111, 7]",[],"[(58, 61), (87, 93)]"
...,...,...,...
16055,[228],[],[]
16133,[7],"[(10, 14), (2795, 2799)]","[(5, 8), (10, 13), (945, 948), (1560, 1563), (..."
16134,[224],"[(19, 23), (2151, 2155)]",[]
16135,[230],"[(4, 8), (10, 14), (2625, 2629)]",[]


In [51]:
train_data = mes_many[(mes_many['tickets_spans'].str.len() != 0) & (mes_many['names_spans'].str.len() != 0)]

In [ ]:
train_data

In [60]:
train_data['MessageText']

2243      Анонс ключевых событий, 20 ноября  В России  -...
3328      Сегодня большой день корпоративной отчетности....
3933      МТС планирует рассмотреть механизмы по возможн...
4881      ВТБ с 25 сентября запустил виртуального мобиль...
5162      Московская биржа по рекомендации индексного ко...
                                ...                        
595637    ​🏦 Московский кредитный банк (CBOM) - обзора р...
595638    ​🏘 Эталон (ETLN) - обзор результатов компании ...
595639    "​♻️ Сегежа (SGZH) - обзор финансовых результа...
595644    "​​🏦 ВТБ (VTBR) - обзор основных финансовых по...
595653    "​​🔥 Газпром (GAZP) - обзор долгосрочных персп...
Name: MessageText, Length: 46736, dtype: object

In [58]:
train_data

,MessageID,ChannelID,DateAdded,DatePosted,MessageText,IsForward,tickets_spans,names_spans
2243,2306,1001029560,2021-02-06 00:52:00,2017-11-20 08:53:02,"Анонс ключевых событий, 20 ноября В России -...",False,"[(47, 50)]","[(64, 71)]"
3328,3419,1001029560,2021-02-06 00:52:00,2018-08-29 18:53:22,Сегодня большой день корпоративной отчетности....,False,"[(830, 833)]","[(300, 307), (558, 572), (967, 970), (1088, 10..."
3933,4028,1001029560,2021-02-06 00:52:00,2019-02-08 16:45:16,МТС планирует рассмотреть механизмы по возможн...,False,"[(198, 202)]","[(0, 3), (143, 146), (458, 461)]"
4881,4980,1001029560,2021-02-06 00:52:00,2019-09-25 13:03:27,ВТБ с 25 сентября запустил виртуального мобиль...,False,"[(646, 649)]","[(0, 3), (62, 65), (140, 143), (326, 329), (39..."
5162,5263,1001029560,2021-02-06 00:52:00,2019-11-27 10:46:45,Московская биржа по рекомендации индексного ко...,False,"[(215, 218), (522, 525), (789, 792)]","[(0, 16), (285, 292), (575, 578), (605, 612), ..."
...,...,...,...,...,...,...,...,...
595637,1399,1565800335,2023-04-07 02:43:04,2023-03-30 07:30:00,​🏦 Московский кредитный банк (CBOM) - обзора р...,False,"[(30, 34)]","[(3, 28)]"
595638,1401,1565800335,2023-04-07 02:43:04,2023-03-31 07:30:00,​🏘 Эталон (ETLN) - обзор результатов компании ...,False,"[(11, 15), (2379, 2383)]","[(639, 650)]"
595639,1403,1565800335,2023-04-07 02:43:04,2023-04-01 07:30:00,"""​♻️ Сегежа (SGZH) - обзор финансовых результа...",False,"[(13, 17), (2830, 2834)]","[(5, 11), (1739, 1745), (2242, 2253), (2822, 2..."
595644,1414,1565800335,2023-04-07 02:43:04,2023-04-06 07:30:56,"""​​🏦 ВТБ (VTBR) - обзор основных финансовых по...",False,"[(10, 14), (2795, 2799)]","[(5, 8), (10, 13), (945, 948), (1560, 1563), (..."


In [33]:
mes_many['MessageText']

0                                                          
1         http://bcs-express.ru/investidei/usdrubukoil/n...
2         http://bcs-express.ru/novosti-i-analitika/inte...
3         http://bcs-express.ru/novosti-i-analitika/priz...
4         http://bcs-express.ru/novosti-i-analitika/kak-...
                                ...                        
595651    🇷🇺 Комментарий по рублю. Что с ним будет дальш...
595652    Все ведь уже слышали про нейросеть GPT-4? Ее п...
595653    "​​🔥 Газпром (GAZP) - обзор долгосрочных персп...
595654    👆 Смотрите! Он показывает подписчикам, как он ...
595655    ​​📊 Как повлияла более жесткая ДКП на экономик...
Name: MessageText, Length: 595656, dtype: object

In [15]:
def extract_text_from_spans(text, spans):
    res = []
    for span in spans:
        res += [text[span[0] : span[1]]]
    return '+'.join(res)

In [20]:
mes_many[['names_spans', 'MessageText']].apply(lambda x : extract_text_from_spans (x['MessageText'], x['names_spans']), axis =1  )

0                                                        
1                                                        
2                                                        
3                                                        
4                                                        
                               ...                       
595651                                                   
595652                                                   
595653    Газпром+Газпром+Газпром+Газпром+Газпром+Новатэк
595654                                                   
595655                                                   
Length: 595656, dtype: object